In [1]:
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
tickers = ["MMM", "AXP", "AAPL", "BA", "CAT", "CVX", "CSCO", "KO", "DD", "XOM", "GE", "GS", "HD", "IBM", "INTC", "JNJ", "JPM", "MCD", "MRK", "MSFT", "NKE", "PFE", "PG", "TRV", "UNH", "RTX", "VZ", "V", "WMT", "DIS"]
data = yf.download(tickers, start="2009-01-01", end="2018-09-30")['Close']

[*********************100%%**********************]  30 of 30 completed


In [3]:
data.head()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,NKE,PFE,PG,RTX,TRV,UNH,V,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-02,3.241071,19.330000,45.250000,46.910000,16.959999,76.519997,21.932587,23.920000,102.504051,86.760002,...,13.2650,17.333965,62.799999,34.581497,45.200001,27.590000,13.3600,32.488476,57.180000,81.639999
2009-01-05,3.377857,19.950001,46.169998,46.080002,17.110001,76.660004,21.420210,23.500000,99.861885,88.779999,...,13.3600,17.229603,62.349998,34.172436,44.529999,27.139999,13.4550,30.462635,56.520000,81.629997
2009-01-06,3.322143,21.070000,46.310001,45.799999,17.790001,77.349998,22.843479,24.309999,101.243019,88.709999,...,13.0900,16.888046,62.169998,34.537445,43.169998,26.500000,14.4025,29.974932,56.020000,80.300003
2009-01-07,3.250357,20.010000,44.759998,43.669998,17.320000,73.959999,22.886177,23.180000,96.739326,84.500000,...,12.6150,16.593927,61.080002,33.681561,41.459999,26.299999,14.0900,29.918659,55.540001,78.250000
2009-01-08,3.310714,20.040001,44.790001,44.020000,17.540001,74.239998,22.501896,22.900000,96.919472,85.410004,...,12.8475,16.745731,60.410000,33.675266,42.230000,26.709999,13.9400,30.378225,51.380001,79.089996


In [4]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)


Expected annual return: 29.3%
Annual volatility: 18.4%
Sharpe Ratio: 1.48


(0.2926688113499274, 0.18437264145307525, 1.4789006069499981)

In [5]:
data_weights = data.copy()
for ticker in tickers:
    data_weights[ticker + '_weight'] = cleaned_weights[ticker]


In [6]:
data_weights

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,NKE_weight,PFE_weight,PG_weight,TRV_weight,UNH_weight,RTX_weight,VZ_weight,V_weight,WMT_weight,DIS_weight
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-02,3.241071,19.330000,45.250000,46.910000,16.959999,76.519997,21.932587,23.920000,102.504051,86.760002,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2009-01-05,3.377857,19.950001,46.169998,46.080002,17.110001,76.660004,21.420210,23.500000,99.861885,88.779999,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2009-01-06,3.322143,21.070000,46.310001,45.799999,17.790001,77.349998,22.843479,24.309999,101.243019,88.709999,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2009-01-07,3.250357,20.010000,44.759998,43.669998,17.320000,73.959999,22.886177,23.180000,96.739326,84.500000,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2009-01-08,3.310714,20.040001,44.790001,44.020000,17.540001,74.239998,22.501896,22.900000,96.919472,85.410004,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-24,55.197498,109.849998,367.989990,154.000000,48.439999,122.620003,97.066994,112.769997,70.497810,232.899994,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2018-09-25,55.547501,109.870003,367.230011,154.089996,48.470001,123.370003,96.711182,113.629997,67.675491,232.500000,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0
2018-09-26,55.105000,108.010002,365.029999,153.169998,48.410000,121.949997,95.202515,115.209999,68.396088,228.880005,...,0.0,0.0,0.0,0.0,0.17714,0.0,0.0,0.20605,0.0,0.0


In [ ]:
train = data[:'2015-01-01']
validation = data['2015-01-01':'2016-01-01']
test = data['2016-01-01':]